<a href="https://colab.research.google.com/github/sharpjaktim-byte/ILKOM-ANALISIS-DATA/blob/main/Scrapping_Data_Google_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title install selenium, Update, Chromium
!pip install selenium
!apt-get update
!pip install aspose-cells-python
!pip install xlsxwriter
!pip install dash
#!apt-get install chromium chromium-driver

import os, glob
import selenium
from selenium import webdriver
import shutil

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from IPython import display
import pathlib
from datetime import datetime

import aspose.cells
from aspose.cells import License, Workbook, FileFormatType
import pandas as pd

from datetime import datetime
import time




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 36.6 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,969 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def web_driver():
    options = webdriver.ChromeOptions()
    prefs = {
        "profile.default_content_settings.popups": 0,
        "download.default_directory": '/content/Data/Outstanding/',
        "directory_upgrade": True
    }
    options.add_experimental_option("prefs", prefs)

    # Set bahasa ke Indonesia
    options.add_argument("--lang=id")
    options.add_argument("--accept-language=id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7")

    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--window-size=1920,1200")
    options.add_argument("--disable-dev-shm-usage")

    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36"
    )

    driver = webdriver.Chrome(options=options)
    return driver


driver = web_driver()

# === URL dengan parameter bahasa Indonesia ===
#url = "https://www.google.com/maps/place/SHARP+Service+Center+Pulogadung/@-6.1892456,106.9232986,17z/data=!4m8!3m7!1s0x2e698b3993dbd903:0x1b5275c7fdf1b852!8m2!3d-6.1892456!4d106.9258789!9m1!1b1!16s%2Fg%2F11ggvxrdyx?hl=id"
url = "https://www.google.com/maps/place/SHARP+Service+Center+Jakarta+Selatan/@-6.2712503,106.7728185,17z/data=!4m8!3m7!1s0x2e69f133087de7f7:0xe64c50e674687e5f!8m2!3d-6.2712503!4d106.7753988!9m1!1b1!16s%2Fg%2F11g0g4kvx4?hl=id&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D"

driver.get(url)
print("[INFO] Membuka halaman Google Maps (bahasa Indonesia)...")
time.sleep(6)

# === Klik tombol 'Lihat semua ulasan' ===
try:
    all_reviews_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((
            By.XPATH,
            "//button[contains(@aria-label,'ulasan') or contains(text(),'Ulasan') or contains(text(),'reviews')]"
        ))
    )
    driver.execute_script("arguments[0].click();", all_reviews_btn)
    print("[INFO] Masuk ke panel ulasan.")
    time.sleep(3)
except Exception as e:
    print("[WARN] Tombol 'Lihat semua ulasan' tidak ditemukan:", e)

# === Tunggu panel ulasan ===
try:
    panel = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf"
        ))
    )
except:


    panel = driver.find_element(By.TAG_NAME, "body")

# === Klik menu "Urutkan" untuk memilih "Yang terbaru" ===
try:
    print("[INFO] Mengatur urutan ulasan: terbaru dulu.")
    sort_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Urutkan') or contains(@aria-label, 'Sort')]"))
    )
    driver.execute_script("arguments[0].click();", sort_button)
    time.sleep(1)

    newest_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(text(),'Terbaru') or contains(text(),'Newest')]"))
    )
    driver.execute_script("arguments[0].click();", newest_option)
    time.sleep(3)
    print("[INFO] ✅ Ulasan sekarang diurutkan dari yang terbaru.")
except Exception as e:
    print("[WARN] ⚠️ Tidak bisa mengubah urutan ulasan otomatis:", e)


reviews_data = []
seen_reviews = set()
scroll_attempt = 0
max_scroll_attempts = 10

print("[INFO] Mulai ambil data ulasan...")

while scroll_attempt < max_scroll_attempts and len(reviews_data) < 100:
    review_containers = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf, div.NMMf9d')

    for container in review_containers:
        # --- Nama pengguna ---
        try:
            username = container.find_element(By.CLASS_NAME, 'd4r55').text.strip()
        except:
            username = 'Unknown'

        # --- Klik "Lihat ulasan asli" kalau muncul ---
        try:
            view_original_btn = container.find_element(
                By.XPATH,
                ".//button[contains(text(),'Lihat ulasan asli') or contains(text(),'View original')]"
            )
            driver.execute_script("arguments[0].click();", view_original_btn)
            time.sleep(0.3)
        except:
            pass

        # --- Klik "Lainnya" / "See more" ---
        try:
            more_btn = container.find_element(
                By.XPATH,
                ".//button[contains(text(),'Lainnya') or contains(text(),'More')]"
            )
            driver.execute_script("arguments[0].click();", more_btn)
            time.sleep(0.3)
        except:
            pass

        # --- Isi ulasan ---
        try:
            review = container.find_element(By.CLASS_NAME, 'wiI7pd').text.strip()
        except:
            review = ''

        # --- Rating bintang ---
        try:
            rating_element = container.find_element(By.CLASS_NAME, 'kvMYJc')
            rating = rating_element.get_attribute('aria-label').split()[0]
        except:
            rating = 'Unknown'

        # --- Waktu review (misal: "6 bulan lalu") ---
        try:
            time_posted = container.find_element(By.CSS_SELECTOR, 'span.rsqaWe, span.p2TkOb').text.strip()
        except:
            time_posted = 'Unknown'

        # --- Simpan data unik ---
        unique_key = (username, review)
        if review and unique_key not in seen_reviews:
            seen_reviews.add(unique_key)
            reviews_data.append({
                'user': username,
                'review': review,
                'rating': rating,
                'time': time_posted
            })

    # Scroll panel
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", panel)
    time.sleep(2)
    scroll_attempt += 1
    print(f"[INFO] Scroll ke-{scroll_attempt}, total ulasan: {len(reviews_data)}")

    if len(reviews_data) >= 100:
        break

# === Simpan hasil ===
df = pd.DataFrame(reviews_data)
df.to_excel('ulasan_sharp_Jaksel.xlsx', index=False)
print(f"[INFO] Total {len(df)} ulasan disimpan ke 'ulasan_sharp_Jaksel.xlsx'.")

driver.quit()


[INFO] Membuka halaman Google Maps (bahasa Indonesia)...
[INFO] Masuk ke panel ulasan.
[INFO] Mengatur urutan ulasan: terbaru dulu.
[INFO] ✅ Ulasan sekarang diurutkan dari yang terbaru.
[INFO] Mulai ambil data ulasan...
[INFO] Scroll ke-1, total ulasan: 5
[INFO] Scroll ke-2, total ulasan: 11
[INFO] Scroll ke-3, total ulasan: 18
[INFO] Scroll ke-4, total ulasan: 23
[INFO] Scroll ke-5, total ulasan: 27
[INFO] Scroll ke-6, total ulasan: 32
[INFO] Scroll ke-7, total ulasan: 38
[INFO] Scroll ke-8, total ulasan: 48
[INFO] Scroll ke-9, total ulasan: 54
[INFO] Scroll ke-10, total ulasan: 60
[INFO] Total 60 ulasan disimpan ke 'ulasan_sharp_Jaksel.xlsx'.


In [ ]:
df

,user,review,rating,time
0,Adit Jaya,Sya mau memesan LED nya hp sharp aqua dimana y...,5,2 bulan lalu
1,Nur Afifah,Bener bener saya kecewa sama pelayanan service...,1,2 bulan lalu
2,Suhaemah Suhaemah,"Bener bener pelayanannya buruk, pantes cabang ...",1,2 bulan lalu
3,Stanly Fredrik Napoleon,pelayanan baik dan ok untuk ukuran service cen...,4,3 bulan lalu
4,Agung wiratno,Kalau mau cari Spareparts Merek Sharps apapun ...,5,3 bulan lalu
5,ramotparulian purba,Layanan yang sistemnya bikin rugi waktu dan ru...,1,4 bulan lalu
6,Asteria Putri,"Pelayanan Sharp buruk bgt,udh hotline nya tdk ...",1,5 bulan lalu
7,Rocket Rockers,"Ada LCD hp ga sih, nyari ko susah banget,, ga ...",1,5 bulan lalu
8,Firdaus Habibi,Boleh minta nomer telfon kantornya ? Yg terter...,3,7 bulan lalu
9,Kurniawan Fahmi,Ternyata teknisi Sharp yang datang supportive ...,4,Diedit 9 bulan lalu


In [ ]:
dis = driver.page_source
display.HTML(data=dis)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')